In [123]:
# Импорт модулей

In [124]:
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from tqdm import tqdm
import cv2
import numpy as np
import psycopg2

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as immg

import torch

import torchvision
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [125]:
# Создание коннекта с БД

In [126]:
conn = psycopg2.connect(database="FCW", user="postgres",
    password="123", host="172.17.0.2", port=5432)

In [127]:
# Импорт данных с БД

In [128]:
cur = conn.cursor()
cur.execute("SELECT * FROM dataset")
df = cur.fetchall()

In [129]:
# Создание датафрейма и проверка пустых значений

In [130]:
data = pd.DataFrame(data=df, columns=["xmin","ymin", "xmax", "ymax", "class", "file", "width", "height"])
print(data)
print(data.isnull().sum())

      xmin  ymin  xmax  ymax         class                file  width  height
0      804   298   860   365  missing_hole  01_missing_hole_05   3034    1586
1      279   294   353   374  missing_hole  01_missing_hole_05   3034    1586
2     1068  1051  1110  1107  missing_hole  01_missing_hole_05   3034    1586
3     1613   392  1677   460  missing_hole  01_missing_hole_05   3034    1586
4     1062  1047  1117  1112  missing_hole  01_missing_hole_12   3034    1586
...    ...   ...   ...   ...           ...                 ...    ...     ...
2948  2714  1178  2740  1204  open_circuit  01_open_circuit_12   3034    1586
2949  1706   587  1732   613  open_circuit  01_open_circuit_12   3034    1586
2950   466  2235   546  2313  open_circuit  04_open_circuit_02   3056    2464
2951   421  1722   502  1779  open_circuit  04_open_circuit_02   3056    2464
2952   128  1228   197  1306  open_circuit  04_open_circuit_02   3056    2464

[2953 rows x 8 columns]
xmin      0
ymin      0
xmax      0
yma

In [131]:
# Разделение датасета на обучающую и тестовую выборку

In [132]:
train, test = train_test_split(data, shuffle=True, test_size=0.2, random_state=34)

In [133]:
# Преобразование категориальных данных

In [134]:
classes_la = {"missing_hole": 0, "mouse_bite": 1, "open_circuit":2, "short": 3, 'spur': 4,'spurious_copper':5}

train["class"] = train["class"].apply(lambda x: classes_la[x])
test["class"] = test["class"].apply(lambda x: classes_la[x])

In [135]:
# Группировка набора данных

In [136]:
df = train.copy()
df_grp = df.groupby(['file'])

In [137]:
# Подготовка изображений и датасета

In [138]:
def plot_image(image_name):
    print(image_name)
    image_group = df_grp.get_group(image_name)
    bbox = image_group.loc[:,['xmin', 'ymin', 'xmax', 'ymax']]
    path ="./PCB_DATASET/images/"
    if "missing" in name.split('_'):
        path += 'Missing_hole/'
    if "mouse" in name.split('_'):
        path += 'Mouse_bite/'
    if "open" in name.split('_'):
        path += 'Open_circuit/'
    if "short" in name.split('_'):
        path += 'Short/'
    if "spur" in name.split('_'):
        path += 'Spur/'
    if "spurious" in name.split('_'):
        path += 'Spurious_copper/'
   
    img = immg.imread(path+""+name+'.jpg')
    fig,ax = plt.subplots(figsize=(18,10))
    ax.imshow(img,cmap='binary')
    for i in range(len(bbox)):
        box = bbox.iloc[i].values
        x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none',)
        ax.add_patch(rect)


In [139]:
# Определение класса для создания тензора

In [140]:
class fcbData(object):
    def __init__(self, df, IMG_DIR, transforms): 
        self.df = df
        self.img_dir = IMG_DIR
        self.image_ids = self.df['file'].unique().tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        a = ''
        if "missing" in image_id.split('_'):
            a = 'Missing_hole/'
        elif "mouse" in image_id.split('_'):
            a = 'Mouse_bite/'
        elif "open" in image_id.split('_'):
            a = 'Open_circuit/'
        elif "short" in image_id.split('_'):
            a = 'Short/'
        elif "spur" in image_id.split('_'):
            a = 'Spur/'
        elif "spurious" in image_id.split('_'):
            a = 'Spurious_copper/'
        image_values = self.df[self.df['file'] == image_id]
        image = cv2.imread(self.img_dir+a+image_id+".jpg",cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        boxes = image_values[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        labels = image_values["class"].values
        labels = torch.tensor(labels)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])
        target['area'] = torch.as_tensor(area, dtype=torch.float32)
        target['iscrowd'] = torch.zeros(len(classes_la), dtype=torch.int64)

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
        
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return torch.tensor(image), target, image_id

In [141]:
def get_train_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [142]:
# Загрузка изображений для преобразования

In [143]:
path ="./PCB_DATASET/images/"
fcb_dataset   = fcbData(df, path, get_train_transform())

In [144]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [145]:
train_dataset = fcbData(df, path, get_train_transform())
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=6,
    collate_fn=collate_fn
)

In [146]:
# Загрузка предобученной модели R-CNN ResNet-50 для обнаружения объектов и классификации

In [147]:
num_classes = 6

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [148]:
# Определение устройства обработки и загрузка оптимизаторов

In [149]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [150]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001, weight_decay=0.0005,)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [151]:
# Установлены 2 эпохи

In [152]:
num_epochs = 2

In [153]:
# Обучение модели

In [154]:
import sys
best_epoch = 1
min_loss = sys.maxsize

for epoch in range(num_epochs):
    tk = tqdm(train_data_loader)
    model.train();
    for images, targets, image_ids in tk:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        tk.set_postfix(train_loss=loss_value)
    tk.close()

    print(f"Epoch #{epoch} loss: {loss_value}") 

  0%|          | 0/690 [00:00<?, ?it/s]

Epoch #0 loss: 0.095889732020452


  0%|          | 0/690 [00:00<?, ?it/s]

Epoch #1 loss: 0.009387831319037344


In [155]:
torch.save(model.state_dict(), 'pcbdetection.pt')

In [156]:
# Оценка

In [157]:
y_true =[]
y_pred = []
for i in range(50):
    img,target,_ = test_dataset[i]
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])[0]
        y_true.append(target['labels'][0])
        y_pred.append(prediction['labels'][0])

In [158]:
yy_pred = []
for v in y_pred:
    yy_pred.append(v.cpu())

In [159]:
# Матрица ошибок

In [160]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, yy_pred)

array([[ 0,  1,  0,  1,  0,  0],
       [ 0,  7,  2,  0,  0,  0],
       [ 0,  0,  9,  0,  0,  0],
       [ 0,  0,  0,  9,  0,  0],
       [ 0,  1,  0,  0,  2,  6],
       [ 0,  0,  0,  0,  0, 12]])

In [161]:
# Отчет о классификации

In [162]:
from sklearn.metrics import classification_report
print(classification_report(y_true, yy_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.78      0.78      0.78         9
           2       0.82      1.00      0.90         9
           3       0.90      1.00      0.95         9
           4       1.00      0.22      0.36         9
           5       0.67      1.00      0.80        12

    accuracy                           0.78        50
   macro avg       0.69      0.67      0.63        50
weighted avg       0.79      0.78      0.73        50

